In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as sk
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import accuracy_score
%matplotlib inline


In [2]:
USE_CUDA=torch.cuda.is_available()
DEVICE=torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
# Epoch과 Batch_size 선언
EPOCHS = 1
BATCH_SIZE = 8


In [4]:
df=pd.read_excel('../optimal_data2/'+'Continous_2weeks_2day_1term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

scaler = StandardScaler()
X = scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]
Count=int(322/10)*83
pred_list=[]

In [5]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.input_layer = nn.Linear(6, 128)
        self.hidden_layer1 = nn.Linear(128, 256)
        self.hidden_layer2 = nn.Linear(256, 128)
        self.output_layer   = nn.Linear(128,3)
        
        # Define proportion or neurons to dropout
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        out =  F.relu(self.input_layer(x))
        out =  F.relu(self.hidden_layer1(out))
        out= self.dropout(out)
        out =  F.relu(self.hidden_layer2(out))
        out= self.dropout(out)
        out =  F.relu(self.output_layer(out))
        return out 

In [6]:
# model=DNN().to(DEVICE)
# # 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
# optimizer    = optim.Adam(model.parameters(), lr=0.01)

# # 손실함수(loss function)을 지정합니다. Multi-Class Classification 이기 때문에 CrossEntropy 손실을 지정하였습니다.
# loss_fn = nn.CrossEntropyLoss()

### 학습하기

In [7]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target=data.to(DEVICE), target.to(DEVICE)
        #매 반복(iteration) 마다 기울기를 계산하기 위해 zero_grad() 호출
        optimizer.zero_grad()
        # 실제 모델의 예측값(output) 받아오기
        output=model(data)
        #정답 데이터와의 CrossEntropyLoss 계산
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = F.cross_entropy(output,target)
        #기울기 계산
        loss.backward()
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()


### 테스트 하기

In [8]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()
    # 필요한 변수 초기화
    # Test과정에서의 Loss = test_loss
    # 실제 모델의 예측이 정답과 맞은 횟수 = correct
    test_loss = 0
    correct = 0
    pred_list=[]
    pred_array=[]
    with torch.no_grad(): # 평가 과정에서는 기울기를 계산하지 않으므로, no_grad명시
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            _, pred = output.max(dim=1)
            pred_array=pred.tolist()
            pred_list+=pred_array
                                    # confusion matrix를 위해 pred 리턴 값
           
            # 모든 오차 더하기
            test_loss +=  F.cross_entropy(output, target, reduction="sum").item()
            
            # 가장 큰 값을 가진 클래스가 모델의 예측입니다.
            # 예측 클래스(pred)과 정답 클래스를 비교하여 일치할 경우 correct에 1을 더합니다.
            pred = output.max(1, keepdim=True)[1]
            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    #정확도 계산
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy, pred_list

In [9]:
#한 사람당 데이터 수
Count_1=int(83*0.1)
#한 사람당 데이터 수
Count_2=83

X_test=pd.DataFrame()
X_train=pd.DataFrame()
y_test=pd.DataFrame()
y_train=pd.DataFrame()
empty=pd.DataFrame()

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]



In [10]:
for i in range(10):
    model=DNN().to(DEVICE)
    # 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
    optimizer    = optim.Adam(model.parameters(), lr=0.01)

    # 손실함수(loss function)을 지정합니다. Multi-Class Classification 이기 때문에 CrossEntropy 손실을 지정하였습니다.
    loss_fn = nn.CrossEntropyLoss()
    X_test=empty
    X_train=empty
    y_test=empty
    y_train=empty
    y_test_list=[]
    for j in range(322):
        X_temp_test=X.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        X_test=pd.concat([X_test,X_temp_test])
        X_temp_train=X.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        X_train=pd.concat([X_train,X_temp_train])
        
        y_temp_test=y.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        y_test=pd.concat([y_test,y_temp_test])
        y_temp_train=y.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        y_train=pd.concat([y_train,y_temp_train])
        
    
    print('SMOTE 적용 전 Train 레이블 값 분포: \n', y_train.value_counts())
    print('SMOTE 적용 전 Test 레이블 값 분포: \n', y_test.value_counts())
    
    # SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)
    print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_test.shape, y_test.shape)
    print('SMOTE 적용 후 Train 레이블 값 분포: \n', y_train.value_counts())
    print('SMOTE 적용 후 Test 레이블 값 분포: \n', y_test.value_counts())

    
    #모든 데이터 torch로 변환
    X_train = torch.FloatTensor(X_train.to_numpy())
    X_test = torch.FloatTensor(X_test.to_numpy())    
    print("X_test",X_test)
    y_train=y_train.to_numpy()
    y_train=np.ravel(y_train, order='C')
    y_train = torch.LongTensor(y_train)
    y_test=y_test.to_numpy()
    y_test=np.ravel(y_test, order='C')
    y_test = torch.LongTensor(y_test)
    
    # train_dataset, test_dataset을 구별하여 정의
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset=TensorDataset(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=16,shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16,shuffle=False)
    

    for epoch in range(1, EPOCHS + 1):
        train(model, train_dataloader, optimizer)
        test_loss, test_accuracy, predict = evaluate(model, test_dataloader)

        print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))
        
#     print("[{}]Predict : {}".format(i,predict))
    #Accuracy
    #test label 데이터 토치에서 list로 변경
    y_test_list=y_test.tolist()

    accuracy=accuracy_score(y_test_list, predict) * 100
    print("[{}]Accuracy : {}".format(i,accuracy))   
    #f1score
    f1 = f1_score(y_test_list,predict, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    #precision/recall
    p_rlist=sk(y_test_list,predict,average='weighted')
    print("[{}]Precision : {}".format(i,p_rlist[0]))
    print("[{}]Recall : {}".format(i,p_rlist[1]))
    print()
     #결과 배열에 넣기
    Result[i][0]=accuracy
    Result[i][1]=f1
    Result[i][2]=p_rlist[0]
    Result[i][3]=p_rlist[1]
    del accuracy
    del f1
    del p_rlist

SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    22300
1.0     1194
2.0      656
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    2290
1.0     161
2.0     125
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (6870, 6) (6870, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    22300
1.0    22300
2.0    22300
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    2290
1.0    2290
2.0    2290
dtype: int64
X_test tensor([[ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        ...,
        [ 0.0092, -0.3196, -0.1974, -0.0420, -0.9486,  0.4706],
        [-0.9457, -0.4543, -0.1974,  0.4466, -0.9486, -0.4250],
        [-1.1409, -0.4082, -0.1974, -0.4301, -0.9486, -1.4577]])
[1] Test Loss: 1.0986, Accuracy: 33.33%
[0]Accuracy : 33.33333333333333
[0]F1score : 0.16666666666666666
[0]Precision : 0.1111111111111111
[0]Recall : 0.3333333333333333



C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    22300
1.0     1194
2.0      656
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    2371
1.0     145
2.0      60
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (7113, 6) (7113, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    22300
1.0    22300
2.0    22300
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    2371
1.0    2371
2.0    2371
dtype: int64
X_test tensor([[ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        ...,
        [-0.9418, -0.2774, -0.1214,  2.4070, -0.1867, -1.0238],
        [ 1.0477,  1.9294,  0.4988, -0.0682,  0.7724,  0.3055],
        [-1.2310, -0.1201, -0.1974,  0.4751, -0.9486, -0.3456]])
[1] Test Loss: 1.0986, Accuracy: 33.33%
[1]Accuracy : 33.33333333333333
[1]F1score : 0.16666666666666666
[1]Precision : 0.1111111111111111
[1]Recall : 0.3333333333333333



C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    22300
1.0     1194
2.0      656
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    2295
1.0     165
2.0     116
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (6885, 6) (6885, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    22300
1.0    22300
2.0    22300
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    2295
1.0    2295
2.0    2295
dtype: int64
X_test tensor([[ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        ...,
        [-0.4793,  0.1635, -0.1974,  0.8485,  1.2766,  0.1434],
        [-0.6724,  0.5479, -0.1974,  0.4386,  1.4757,  0.0615],
        [ 0.7879,  0.2913, -0.1974,  1.0074,  0.7211,  1.2165]])
[1] Test Loss: 1.0986, Accuracy: 33.33%
[2]Accuracy : 33.33333333333333
[2]F1score : 0.16666666666666666
[2]Precision : 0.1111111111111111
[2]Recall : 0.3333333333333333



...

C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    22300
1.0     1194
2.0      656
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    2394
1.0     126
2.0      56
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (7182, 6) (7182, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    22300
1.0    22300
2.0    22300
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    2394
1.0    2394
2.0    2394
dtype: int64
X_test tensor([[ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        ...,
        [-0.6015,  0.5152, -0.1974, -0.1047,  1.2600,  0.0590],
        [ 0.7877, -0.1528, -0.1974,  0.5851,  0.1632,  1.1787],
        [ 0.7433,  0.0047, -0.1974,  0.0959,  1.1350,  1.3195]])
[1] Test Loss: 1.0986, Accuracy: 33.33%
[8]Accuracy : 33.33333333333333
[8]F1score : 0.16666666666666666
[8]Precision : 0.1111111111111111
[8]Recall : 0.3333333333333333



C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    22300
1.0     1194
2.0      656
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    2421
1.0      82
2.0      73
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (7263, 6) (7263, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    22300
1.0    22300
2.0    22300
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    2421
1.0    2421
2.0    2421
dtype: int64
X_test tensor([[ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        [ 0.2918, -0.4551, -0.1974, -0.4948, -0.9486, -0.0756],
        ...,
        [-0.7423,  0.1045, -0.0701,  0.4339,  0.4984,  0.1436],
        [-0.3977,  2.0762,  2.1854,  0.0365,  0.9681,  0.0041],
        [-0.2134,  0.3952, -0.1974,  0.0691, -0.9486,  0.6710]])
[1] Test Loss: 1.0986, Accuracy: 33.33%
[9]Accuracy : 33.33333333333333
[9]F1score : 0.16666666666666666
[9]Precision : 0.1111111111111111
[9]Recall : 0.3333333333333333



C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

,Accuracy,F1-Score,Precision,Recall
0,33.333333,0.166667,0.111111,0.333333
1,33.333333,0.166667,0.111111,0.333333
2,33.333333,0.166667,0.111111,0.333333
3,33.333333,0.166667,0.111111,0.333333
4,33.333333,0.166667,0.111111,0.333333
5,33.333333,0.166667,0.111111,0.333333
6,33.333333,0.166667,0.111111,0.333333
7,33.333333,0.166667,0.111111,0.333333
8,33.333333,0.166667,0.111111,0.333333
9,33.333333,0.166667,0.111111,0.333333


In [12]:
Matrix=pd.DataFrame(Result_df['Accuracy'],columns=['Accuracy'])
Matrix['Accuracy']=Result_df['Accuracy']
A=[Result_df['Accuracy'].mean(),Result_df['F1-Score'].mean(),Result_df['Precision'].mean(),Result_df['Recall'].mean()]
A=pd.DataFrame(A,columns=['Accuracy'])
Matrix=pd.concat([Matrix,A])
Matrix=Matrix.transpose()
Matrix.to_excel('./PFMatrix2.xlsx')